# Setup

In [48]:
%cd /kaggle/working/
%rm -rf mlproduction
!git clone https://github.com/daoanhkhoa123/mlproduction.git
%cd /kaggle/working/mlproduction
!ls

/kaggle/working
Cloning into 'mlproduction'...
remote: Enumerating objects: 425, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 425 (delta 61), reused 78 (delta 34), pack-reused 317 (from 1)
Receiving objects: 100% (425/425), 3.60 MiB | 10.86 MiB/s, done.
Resolving deltas: 100% (249/249), done.
/kaggle/working/mlproduction
main.py  pyproject.toml  README.md  runtime  scripts  sqls  src  uv.lock


In [49]:
!chmod +x scripts/*.sh
!scripts/cloud_setup.sh

Resolved 69 packages in 342ms                                        
# This file was autogenerated by uv via the following command:
#    uv pip compile pyproject.toml -o requirements.txt --extra-index-url https://pypi.org/simple --emit-index-url
--index-url https://pypi.org/simple
--extra-index-url https://pypi.org/simple

aiohappyeyeballs==2.6.1
    # via aiohttp
aiohttp==3.13.2
    # via fsspec
aiosignal==1.4.0
    # via aiohttp
annotated-types==0.7.0
    # via pydantic
attrs==25.4.0
    # via aiohttp
certifi==2025.11.12
    # via requests
charset-normalizer==3.4.4
    # via requests
filelock==3.20.1
    # via
    #   huggingface-hub
    #   torch
    #   transformers
frozenlist==1.8.0
    # via
    #   aiohttp
    #   aiosignal
fsspec==2025.12.0
    # via
    #   huggingface-hub
    #   lightning
    #   pytorch-lightning
    #   torch
hf-xet==1.2.0
    # via huggingface-hub
huggingface-hub==0.36.0
    # via
    #   tokenizers
    #   transformers
idna==3.11
    # via
    #   reque

# Run

In [51]:
from runtime.ultils import gdown_folder
gdown_folder()

Retrieving folder contents


Retrieving folder 1aoddGUNcsdJuD6KtSRigVnqLW4uqeHin .ipynb_checkpoints


Retrieving folder contents completed
Building directory structure
Building directory structure completed


Processing file 1pQevQ6Ors5DVuAXLxXiWkGh9LGi63DjE hallu_classes.csv
Processing file 1BWfYcKTw1lBcEpeINvcsL2sW36hsadYT hallu_data.csv


Downloading...
From: https://drive.google.com/uc?id=1pQevQ6Ors5DVuAXLxXiWkGh9LGi63DjE
To: /kaggle/working/mlproduction/data/hallu_classes.csv
100%|██████████| 25.0/25.0 [00:00<00:00, 90.5kB/s]
Downloading...
From: https://drive.google.com/uc?id=1BWfYcKTw1lBcEpeINvcsL2sW36hsadYT
To: /kaggle/working/mlproduction/data/hallu_data.csv
100%|██████████| 10.7M/10.7M [00:00<00:00, 58.9MB/s]
Download completed


Folder downloaded successfully.


# What is the dog doing?


In [52]:
import dagshub
dagshub.init(repo_owner='ollamagena', repo_name='mlpro', mlflow=True)


Initialized MLflow to track repo "ollamagena/mlpro"

Repository ollamagena/mlpro initialized!

# Run

In [53]:
import pandas as pd
from src.dataloaders.ultils import train_test_split_df, load_label_enocder

df = pd.read_csv("data/hallu_data.csv")
df.head()
le = load_label_enocder("data/hallu_classes.csv")
X_train, X_test, y_train, y_test = train_test_split_df(df, "label", le, 0.3, verbose=True)

X_train shape: (4900, 4)
X_test shape: (2100, 4)
y_train shape: (4900,)
y_test shape: (2100,)

Sample rows from X_train:
                                        id  \
4920  c362ad9b-e8c4-4147-964c-20ea7f4b305b   
2944  9874108d-6a58-461e-9514-4122cfd37d58   

                                                context  \
4920  Về lịch sử, Cuba luôn có tỷ lệ giáo dục và biế...   
2944  Bốn quy tắc súc tích và tổng quát cho nghiên c...   

                                                 prompt  \
4920  Trong thời gian tham gia học tập, học sinh tại...   
2944            Newt0n dã hoan thàn phuong phap của ai?   

                                               response  
4920  Học sinh tại Cuba phải tự chi trả cho đồng phụ...  
2944  Newton đã hoàn thiện phương pháp thực nghiệm c...  

Sample values from y_train: [1 2]


In [54]:
import mlflow

mlflow.set_experiment("embedding-sklearn2")


<Experiment: artifact_location='mlflow-artifacts:/9fc02da896174a8d950e6b059532f67a', creation_time=1766052189411, experiment_id='1', last_update_time=1766052189411, lifecycle_stage='active', name='embedding-sklearn2', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [55]:
from src.models.preprop_svm import Preprop, prep_remove_common_words_with_counting, agg_differential, agg_concatenate
from mlflow.models import infer_signature
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
import numpy as np


with mlflow.start_run(run_name="full run with vtack aggeration and bagging SVC"):
    preprop_params = {
        "prepropfn": prep_remove_common_words_with_counting,
        "aggfn": agg_concatenate,
        "encoder_name": "bkai-foundation-models/vietnamese-bi-encoder"
    }

    base_svc_params = {
    "C": 1.0,
    "kernel": "rbf",
    "probability": False,
    "random_state": 42,
    "decision_function_shape": "ovr",
}

    bagging_params = {
        "n_estimators": 10,
        "max_samples": 0.8,
        "max_features": 1.0,
        "bootstrap": True,
        "n_jobs": -1,
        "random_state": 42,
    }


    mlflow.log_params({
        **preprop_params,
        **{f"base_svc.{k}": v for k, v in base_svc_params.items()},
        **{f"bagging.{k}": v for k, v in bagging_params.items()},
    })


    preprop = Preprop(**preprop_params)
    x_train_emd = preprop.call_df(X_train, column1="context", column2="response", batch_size= 8)
    base_svc = SVC(**base_svc_params)
    model = BaggingClassifier(estimator=base_svc, **bagging_params)
    model.fit(x_train_emd, y_train)

    x_test_emd = preprop.call_df(X_test, column1="context", column2="response", batch_size= 8)
    y_pred = model.predict(x_test_emd)

    # Calculate and log metrics
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, average="weighted"),
        "recall": recall_score(y_test, y_pred, average="weighted"),
        "f1_score": f1_score(y_test, y_pred, average="weighted"),
    }
    mlflow.log_metrics(metrics)    

    signature = infer_signature(x_test_emd, model.predict(x_test_emd))
    mlflow.sklearn.log_model(
        sk_model = model,
        name="svc",
        signature=signature,
        input_example=x_train_emd[:5]
    )


ImportError: cannot import name 'agg_concatenate' from 'src.models.preprop_svm' (/kaggle/working/mlproduction/src/models/preprop_svm.py)